# Data Prior to augmenting

In [1]:
!pip install lexicalrichness==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lexicalrichness: filename=lexicalrichness-0.5.0-py3-none-any.whl size=15534 sha256=9c77887197138828f0bf6279921b0a3a4c40444dcf63f272fec9eedf78717680
  Stored in directory: /root/.cache/pip/wheels/c7/c3/9e/b44fe0d801947034ab1a0670323cc04636361c6c17b2708066
Successfully built lexicalrichness
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import matplotlib as plt
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import Counter
from lexicalrichness import LexicalRichness

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
#read dataa into a dataFrame
data = pd.read_csv("/work/hate_tweets.csv")
df = pd.DataFrame(data) 

## Frequency of words

In [4]:
#see the most common words in the data  

#changes number of bars on the graph, and the number of the top words that are counted
TOP_WORDS_TO_GRAB = 100

# Convert tweet text to string data type
df["tweet"] = df["tweet"].astype(str)

#count all of the unique words and their requency
counters1 = pd.DataFrame(Counter(" ".join(df["tweet"]).split()).most_common(TOP_WORDS_TO_GRAB))
counters1.rename(columns = {0 : "word", 1 : "freq"}, inplace = True)

## Word length

In [5]:
listOfTweets = df['tweet'].tolist()
listOfListToken = [re.split(' ', s) for s in listOfTweets]

#for every sentence in the list of tweets, split the words into more list and get a full average of the length per word
totalAvg = 0
for sentence in listOfTweets:
    words = sentence.split()
    currentAvg = sum(map(len, words))/len(words)
    totalAvg += currentAvg

totalAvg /= len(listOfTweets)
totalAvgWorLength1 = totalAvg
print(f"Average length of a word: {totalAvg}")

Average length of a word: 5.458515511526774


## Lexical Diversity & Density Averages of each tweet

In [6]:
#https://textinspector.com/lexical-density-vs-lexical-diversity/  more useful to have ttr for shorter text

#FOR ALL TWEETS AS ONE STRING
#combine all tweets into one string
allTweetsInOneString = ""
for sentence in listOfTweets:
    allTweetsInOneString += sentence

#create a lexobj
lexObj1 = LexicalRichness(allTweetsInOneString)


#FOR ALL TWEETS AS THERE OWN STRING
def ttr(text):
    lex = LexicalRichness(text)
    return lex.ttr

def terms(text):
    lex = LexicalRichness(text)
    return lex.terms

def words(text):
    lex = LexicalRichness(text)
    return lex.words

#create new col's for the df
df['mtld'] = df['tweet'].apply(ttr)
df['terms'] = df['tweet'].apply(terms)
df['words'] = df['tweet'].apply(words)

ttr1 = df['words'].mean()
terms1 = df['terms'].mean()
words1 = df['mtld'].mean()


## TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords

# clean up text (remove 'the', 'and', etc)
stop_words = set(stopwords.words('english'))
df['clean_text'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in stop_words]))

# convert text to numeric stuff, make matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['clean_text'])

# getwords in each tweet
feature_names = tfidf.get_feature_names_out()

# put scores into array
tfidf_scores = []

# print tf/idf scores for every tweet
for i, row in enumerate(tfidf_matrix.toarray()):
    print(f"TWEET {i+1}:")
    for j, score in enumerate(row):
        if score > 0:
            print(f"\t{feature_names[j]}: {score}")

	lil: 0.3191739285573094
	lol: 0.23682033846849232
	nicca: 0.3144413692631065
	redfernave: 0.41189538300756223
	rt: 0.13991296370389106
	steelers: 0.41189538300756223
TWEET 1322:
	8220: 0.14874449420715108
	8221: 0.14757537678475852
	baby: 0.2205856334064655
	blue: 0.2452997176956527
	cutest: 0.2700138019848399
	done: 0.21638609259797997
	ever: 0.19370998508644566
	gahdamn: 0.2700138019848399
	good: 0.18141473656844198
	huhwhatsacondom: 0.2700138019848399
	ivy: 0.2700138019848399
	job: 0.2205856334064655
	moesdeph: 0.2700138019848399
	mommy: 0.2700138019848399
	nigger: 0.12811870293933644
	rt: 0.09171851114427503
	still: 0.18141473656844198
	thing: 0.2126294753343618
	ugly: 0.1722894610739457
	well: 0.19587154911727836
	wooooow: 0.2700138019848399
TWEET 1323:
	fag: 1.0
TWEET 1324:
	faggot: 0.6283505589062353
	fucking: 0.7779303150811272
TWEET 1325:
	border: 0.3186483590244463
	caught: 0.35276291150996786
	co: 0.16880641577198946
	crossing: 0.37271864544288175
	gets: 0.3186483590244463


## 

# Data Post Augmenting

In [8]:
#read dataa into a dataFrame
data = pd.read_csv("/work/augmented_hate_tweets.csv")
df2 = pd.DataFrame(data) 

## Frequency of Word

In [9]:
#see the most common words in the data  

#changes number of bars on the graph, and the number of the top words that are counted
TOP_WORDS_TO_GRAB = 100

#count all of the unique words and their requency
counters2 = pd.DataFrame(Counter(" ".join(df2["0"]).split()).most_common(TOP_WORDS_TO_GRAB))
counters2.rename(columns = {0 : "word", 1 : "freq"}, inplace = True)

## Word Length

In [10]:
listOfTweets = df2['0'].tolist()
listOfListToken = [re.split(' ', s) for s in listOfTweets]

#for every sentence in the list of tweets, split the words into more list and get a full average of the length per word
totalAvg = 0
for sentence in listOfTweets:
    words = sentence.split()
    currentAvg = sum(map(len, words))/len(words)
    totalAvg += currentAvg

totalAvg /= len(listOfTweets)
totalAvgWorLength2 = totalAvg

## Lexical Diversity & Density 

In [11]:
#https://textinspector.com/lexical-density-vs-lexical-diversity/  more useful to have ttr for shorter text

#FOR ALL TWEETS AS ONE STRING
#combine all tweets into one string
allTweetsInOneString = ""
for sentence in listOfTweets:
    allTweetsInOneString += sentence

#create a lexobj
lexObj2 = LexicalRichness(allTweetsInOneString)

def words(text):
    lex = LexicalRichness(text)
    return lex.words

#create new col's for the df
df['mtld'] = df2['0'].apply(ttr)
df['terms'] = df2['0'].apply(terms)
df['words'] = df2['0'].apply(words)

ttr2 = df['words'].mean()
terms2 = df['terms'].mean()
words2 = df['mtld'].mean()

## TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords

# clean up text (remove 'the', 'and', etc)
stop_words = set(stopwords.words('english'))
df['clean_text'] = df2['0'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in stop_words]))

# convert text to numeric stuff, make matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['clean_text'])

# getwords in each tweet
feature_names = tfidf.get_feature_names_out()

# put scores into array
tfidf_scores = []

# print tf/idf scores for every tweet
for i, row in enumerate(tfidf_matrix.toarray()):
    print(f"TWEET {i+1}:")
    for j, score in enumerate(row):
        if score > 0:
            print(f"\t{feature_names[j]}: {score}")

	done: 0.20849510990147396
	ever: 0.18513562977389222
	gahdamn: 0.25521407015663755
	good: 0.1744035460083515
	huhwhatsacondom: 0.25521407015663755
	ivy: 0.25521407015663755
	job: 0.20849510990147396
	moesdeph: 0.25521407015663755
	mommy: 0.25521407015663755
	nigger: 0.12016451281815635
	problem: 0.2181901701180794
	rt: 0.08808123872177265
	still: 0.179344171950526
	thing: 0.20452575020710298
	ugly: 0.16877372209740638
	well: 0.18963571864376402
	wooooow: 0.25521407015663755
TWEET 1323:
	axerophthol: 0.8619528495302184
	fag: 0.506988446798087
TWEET 1324:
	faggot: 0.3942853825000184
	fucking: 0.49176290678848344
	poof: 0.7763428885832313
TWEET 1325:
	border: 0.2953525128470878
	caught: 0.3077027552848993
	co: 0.15006003806663407
	crossing: 0.32510944435152034
	flavor: 0.32510944435152034
	gets: 0.2857729205781721
	http: 0.15230038171392324
	ilovebamf: 0.32510944435152034
	interbreed: 0.32510944435152034
	j3uf1tyubo: 0.32510944435152034
	like: 0.1439652295486559
	looks: 0.271328107731653

# Conclusion

## Frequency of words

In [13]:
DeepnoteChart(counters1, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"word","scale":{"type":"linear"},"stack":"zero"},"y":{"sort":null,"type":"quantitative","field":"freq","scale":{"type":"linear"},"stack":"zero","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"nominal","field":"word","scale":{"scheme":"tableau10"}}}}]}],"title":"Before Augmenting","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [14]:
DeepnoteChart(counters2, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"axis":{"title":"word"},"sort":null,"type":"nominal","field":"word","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"freq","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"nominal","field":"word","scale":{"scheme":"tableau10"}}}}]}],"title":"Post Augmenting","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

## Word Length

In [15]:
print(f"Average length of a word: {totalAvgWorLength1}")
print(f"Average length of a word: {totalAvgWorLength2}")

Average length of a word: 5.458515511526774
Average length of a word: 5.513387412133102


## Lexical Diversity & Density 

In [16]:
print("OF WHOLE TEXT")
print("-------------")
print(f"before: Number of Unique words {(lexObj1.terms)}")
print(f"after : Number of Unique words {(lexObj2.terms)}")
print(f"before: Messure of Lexical diversity via Type-Token Ratio: {lexObj1.ttr}") #calculated via  The number of lexical items/The total number of words * 100
print(f"after : Messure of Lexical diversity via Type-Token Ratio: {lexObj2.ttr}") #calculated via  The number of lexical items/The total number of words * 100


print("AVG ACROSS ALL TWEETS")
print("--------------------")
print(f"before: Avg Number of Words per tweet: {ttr1}")
print(f"after : Avg Number of Words per tweet: {ttr2}")
print(f"before: Avg Number of Unique words per tweet: {terms1}")
print(f"after : Avg Number of Unique words per tweet: {terms2}")
print(f"before: Avg of Messure of Lexical diversity via Type-Token Ratio across all tweets: {terms1}")
print(f"after : Avg of Messure of Lexical diversity via Type-Token Ratio across all tweets: {terms2}")

OF WHOLE TEXT
-------------
before: Number of Unique words 5080
after : Number of Unique words 5496
before: Messure of Lexical diversity via Type-Token Ratio: 0.24912951792457458
after : Messure of Lexical diversity via Type-Token Ratio: 0.274951223172745
AVG ACROSS ALL TWEETS
--------------------
before: Avg Number of Words per tweet: 14.567832167832167
after : Avg Number of Words per tweet: 14.273426573426573
before: Avg Number of Unique words per tweet: 13.690909090909091
after : Avg Number of Unique words per tweet: 13.462237762237763
before: Avg of Messure of Lexical diversity via Type-Token Ratio across all tweets: 13.690909090909091
after : Avg of Messure of Lexical diversity via Type-Token Ratio across all tweets: 13.462237762237763


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e803e9f0-f341-4e63-9e5c-ecfa91ec1687' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>